In [25]:
import pandas

In [26]:
from dataclasses import dataclass, field
from typing import Union, List, Tuple

@dataclass
class EGD():
    type: int
    version: int
    requestid: int
    producerid: str
    exchangeid: int
    timestamp: str
    status: int
    configsignature: int
    reserved: int
    data: str = field(repr=False)

    def init_egd(frame):
        def _get_number(a, num):
            result = 0
            for i in range(num):
                result += (int(a[i]) << 8*i)
            return result
    
        def _get_egd_time(a):
            result = f'{_get_number(a[:4], 4)}.{_get_number(a[4:8], 4)}'
            return result

        def _get_ip(a):
            result = ''
            for i in range(4):
                result += str(a[i])
                result += '.' if i != 3 else ''
        
        tp = _get_number(frame[0:1], 1)
        vr = _get_number(frame[1:2], 1)
        rid = _get_number(frame[2:4], 2)
        pip = _get_ip(frame[4:8])
        exid = _get_number(frame[8:12], 4)
        time = _get_egd_time(frame[12:20])
        st = _get_number(frame[20:24], 4)
        cs = _get_number(frame[24:28], 4)
        rv = _get_number(frame[28:32], 4)
        return EGD(tp, vr, rid, pip, exid, time, st, cs, rv, frame[32:])

    def get_float(frame, start):
        import struct
        return struct.unpack('f', frame[start:start+4])[0]
        

@dataclass(order=True)
class Packet():
    time: float
    ip: str
    size: int = field(compare=False)
    port: int = field(compare=False, repr=False)
    egd: EGD = field(compare=False, repr=False)


class PacketList(List[Packet]):
    def _get_time(self, arr):
        result = 0
        for i in range(8):
            result += (arr[i] << 8*i)
        return result/1000
    
    def _get_ip(self, arr):
        result = ''
        for i in range(4):
            result += str(arr[11-i])
            result += '.' if i != 3 else ''
        return result
    
    def _get_port(self, arr):
        result = 0
        for i in range(2):
            result += (arr[12+i] << 8*i)
        return result
    
    def _get_size(self, arr):
        result = 0
        for i in range(4):
            result += (arr[14+i] << 8*i)
        return result

    def __init__(self, file_name, skip=0):
        offset = 0
        with open(file_name, "rb") as f:
            i = -1
            while True:
                header = f.read(18)
                if len(header) != 18:
                    break
                egd_frame = f.read(self._get_size(header))
                egd = EGD.init_egd(egd_frame)
                i += 1
                if i < skip:
                    continue
                self.append(Packet(self._get_time(header), self._get_ip(header), self._get_size(header), self._get_port(header), egd))

In [27]:
packets = PacketList("тпу 20200628-023339_datafile.raw")

In [30]:
packets[1000].egd

EGD(type=13, version=1, requestid=65318, producerid=None, exchangeid=1, timestamp='1593293625.667793000', status=2, configsignature=2560, reserved=0)

In [31]:
packets[1000].time

1593293624.756

In [16]:
def get_analog_from_tpu(packets):
    tpu = []
    e31 = []
    import struct
    for packet in packets:
        dt = packet.egd
        egd = packet.egd.data
        if dt.exchangeid == 1 and packet.ip == "192.168.101.110":
            param = {
                'SPDSP': EGD.get_float(egd, 20),
                'SPDFB': EGD.get_float(egd, 24),
                'ISP': EGD.get_float(egd, 28),
                'IFB': EGD.get_float(egd, 32),
                'LF': EGD.get_float(egd, 36),
                'SACMD': EGD.get_float(egd, 40),
                'LACMD': EGD.get_float(egd, 44),
                'FACMD': EGD.get_float(egd, 48),
                'LV': EGD.get_float(egd, 56),
                'SV': EGD.get_float(egd, 60),
                'EXREF': EGD.get_float(egd, 64),
            }
            tpu.append(param)
        if dt.exchangeid == 1 and packet.ip in ("192.168.101.116", "192.168.101.117"):
            param = {
                'Ig': EGD.get_float(egd, 12)*1000,
                'Hz': EGD.get_float(egd, 16),
                'If': EGD.get_float(egd, 24),
                'PF': EGD.get_float(egd, 28),
                'Qg': EGD.get_float(egd, 32),
                'Uf': EGD.get_float(egd, 36),
                'Ug': EGD.get_float(egd, 40)*1000,
                'Pg': EGD.get_float(egd, 44),
                'XFR': EGD.get_float(egd, 48),
            }
            e31.append(param)
    return (pandas.DataFrame(tpu), pandas.DataFrame(e31))

In [18]:
tpu, ex31 = get_analog_from_tpu(packets)

In [19]:
tpu

,SPDSP,SPDFB,ISP,IFB,LF,SACMD,LACMD,FACMD,LV,SV,EXREF
0,0.03495,0.0,0.0,0.0,0.00465,-0.93915,-1.0,0.0,0.0,0.00665,0.0
1,0.03495,0.0,0.0,0.0,0.00885,-0.93925,-1.0,0.0,0.0,0.00380,0.0
2,0.03495,0.0,0.0,0.0,0.00665,-0.93950,-1.0,0.0,0.0,0.00170,0.0
3,0.03495,0.0,0.0,0.0,0.00665,-0.93955,-1.0,0.0,0.0,0.00210,0.0
4,0.03495,0.0,0.0,0.0,0.00885,-0.93955,-1.0,0.0,0.0,0.00210,0.0
...,...,...,...,...,...,...,...,...,...,...,...
50181,0.03495,0.0,0.0,0.0,0.00885,-0.93950,-1.0,0.0,0.0,0.00225,0.0
50182,0.03495,0.0,0.0,0.0,0.00885,-0.93950,-1.0,0.0,0.0,0.00215,0.0
50183,0.03495,0.0,0.0,0.0,0.01095,-0.93950,-1.0,0.0,0.0,0.00425,0.0
50184,0.03495,0.0,0.0,0.0,0.00665,-0.93945,-1.0,0.0,0.0,0.00275,0.0


In [20]:
ex31[6490:7647]

,Ig,Hz,If,PF,Qg,Uf,Ug,Pg,XFR
6490,83.069392,6.602219,358.169342,0.0,0.032584,189.319656,995.263934,-0.015021,9.187988
6491,145.420656,6.761149,356.045013,0.0,0.054574,185.113388,1045.855761,-0.066147,8.885096
6492,92.286453,6.489810,379.867340,0.0,-0.007857,179.824387,1099.603295,-0.054136,8.577895
6493,46.913482,6.582347,380.050385,0.0,0.011730,173.947266,1141.222239,-0.021652,8.292463
6494,134.495184,6.647633,402.607574,0.0,0.066273,168.728867,1171.468973,-0.033954,8.018107
...,...,...,...,...,...,...,...,...,...
7642,183.899820,45.168789,187.885849,0.0,1.235977,44.852425,4258.404732,-0.013653,9.999999
7643,174.227074,45.234348,212.059662,0.0,1.155622,44.697227,4248.534679,0.039132,9.999999
7644,4.343331,45.305950,163.895462,0.0,0.012561,39.829433,4250.158787,0.012991,9.999999
7645,6.283407,0.000000,-4.107634,0.0,0.011102,11.038867,3882.927656,-0.024766,0.000000


In [21]:
pandas.plotting.boxplot(ex31)

<Axes: >

In [22]:
ex31.plot()

<Axes: >

In [23]:
tpu.plot()

<Axes: >

In [37]:
pandas.read_csv('E31 EGD Network Report.csv')

,Signal Name,Point Name,Point Offset,Direction,Data Type,Cimplicity Resource,Scale Enabled,Scale Name,Units,Producer ID,Period (ms),Exchange Name,Exchange Length,Exchange Number,Exchange Revision,Destination,Description
0,E31\G31\AUTO_FB,EGD0000.0,0.0,WRITE,BOOL,E31,Disabled,NaN,NaN,E31,320,G31,60,1,13.0,Broadcast to 192.168.101.255,Exciter is in Auto Regulator
1,E31\G31\BAD_4_2OMA,EGD0000.1,0.1,WRITE,BOOL,E31,Disabled,NaN,NaN,E31,320,G31,60,1,13.0,Broadcast to 192.168.101.255,Analog Input signal is bad
2,E31\G31\FLD_GRND,EGD0000.2,0.2,WRITE,BOOL,E31,Disabled,NaN,NaN,E31,320,G31,60,1,13.0,Broadcast to 192.168.101.255,Field Gnd Detected
3,E31\G31\M1_BRIDGE_AL,EGD0000.3,0.3,WRITE,BOOL,E31,Disabled,NaN,NaN,E31,320,G31,60,1,13.0,Broadcast to 192.168.101.255,M1 Bridge 1 Alarm
4,E31\G31\M1_BRDGE_TRP,EGD0000.4,0.4,WRITE,BOOL,E31,Disabled,NaN,NaN,E31,320,G31,60,1,13.0,Broadcast to 192.168.101.255,M1 Bridge 1 Tripped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,UDH\E31\HIST\HEALTHY,EGD0000.2,0.2,READ,BOOL,NaN,Disabled,NaN,NaN,NaN,320,E31:LINK,0,0,1.0,NA - Link Health,NaN
102,UDH\G31\EX4\HEALTHY,EGD0000.3,0.3,READ,BOOL,NaN,Disabled,NaN,NaN,NaN,320,E31:LINK,0,0,1.0,NA - Link Health,NaN
103,UDH\G31\EX2\HEALTHY,EGD0000.4,0.4,READ,BOOL,NaN,Disabled,NaN,NaN,NaN,320,E31:LINK,0,0,1.0,NA - Link Health,NaN
104,UDH\G31\EX5\HEALTHY,EGD0000.5,0.5,READ,BOOL,NaN,Disabled,NaN,NaN,NaN,320,E31:LINK,0,0,1.0,NA - Link Health,NaN


In [36]:
pandas.read_csv('X30 EGD Network Report.csv')

,Signal Name,Point Name,Point Offset,Direction,Data Type,Cimplicity Resource,Scale Enabled,Scale Name,Units,Producer ID,Period (ms),Exchange Name,Exchange Length,Exchange Number,Exchange Revision,Destination,Description
0,X30\S30\AUTO_FB,EGD0000.0,0.0,WRITE,BOOL,X30,Disabled,NaN,NaN,X30,320,S30,84,1,12.1,Broadcast to 192.168.101.255,Exciter is in Auto Regulator
1,X30\S30\BAD_4_2OMA,EGD0000.1,0.1,WRITE,BOOL,X30,Disabled,NaN,NaN,X30,320,S30,84,1,12.1,Broadcast to 192.168.101.255,Analog Input signal is bad
2,X30\S30\FLD_GRND,EGD0000.2,0.2,WRITE,BOOL,X30,Disabled,NaN,NaN,X30,320,S30,84,1,12.1,Broadcast to 192.168.101.255,Field Gnd Detected
3,X30\S30\M1_BRIDGE_AL,EGD0000.3,0.3,WRITE,BOOL,X30,Disabled,NaN,NaN,X30,320,S30,84,1,12.1,Broadcast to 192.168.101.255,M1 Bridge 1 Alarm
4,X30\S30\M1_BRDGE_TRP,EGD0000.4,0.4,WRITE,BOOL,X30,Disabled,NaN,NaN,X30,320,S30,84,1,12.1,Broadcast to 192.168.101.255,M1 Bridge 1 Tripped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,UDH\X30\S30\HEALTHY,EGD0000.1,0.1,READ,BOOL,NaN,Disabled,NaN,NaN,NaN,320,X30:LINK,0,0,1.0,NA - Link Health,NaN
93,UDH\X30\VMAX1\HEALTHY,EGD0000.2,0.2,READ,BOOL,NaN,Disabled,NaN,NaN,NaN,320,X30:LINK,0,0,1.0,NA - Link Health,NaN
94,UDH\S30\EX2\HEALTHY,EGD0000.3,0.3,READ,BOOL,NaN,Disabled,NaN,NaN,NaN,320,X30:LINK,0,0,1.0,NA - Link Health,NaN
95,UDH\S30\EX7\HEALTHY,EGD0000.4,0.4,READ,BOOL,NaN,Disabled,NaN,NaN,NaN,320,X30:LINK,0,0,1.0,NA - Link Health,NaN


In [44]:
pandas.read_csv('L31 EGD Network Report.csv')

,Signal Name,Point Name,Point Offset,Direction,Data Type,Cimplicity Resource,Scale Enabled,Scale Name,Units,Producer ID,Period (ms),Exchange Name,Exchange Length,Exchange Number,Exchange Revision,Destination,Description
0,L31\L30SS,EGD0000.0,0.0,WRITE,BOOL,L31,Disabled,NaN,NaN,192.168.101.110,50,FDBKS,68,1,10.0,Broadcast to 192.168.101.255,STATIC START (LCI) ALARM
1,L31\L3SS_RS,EGD0000.1,0.1,WRITE,BOOL,L31,Disabled,NaN,NaN,192.168.101.110,50,FDBKS,68,1,10.0,Broadcast to 192.168.101.255,STATIC STARTER READY TO START
2,L31\L4SS_RUN,EGD0000.2,0.2,WRITE,BOOL,L31,Disabled,NaN,NaN,192.168.101.110,50,FDBKS,68,1,10.0,Broadcast to 192.168.101.255,STATIC STARTER RUNNING
3,L31\LSS_CONNECT,EGD0000.3,0.3,WRITE,BOOL,L31,Disabled,NaN,NaN,192.168.101.110,50,FDBKS,68,1,10.0,Broadcast to 192.168.101.255,NaN
4,L31\TORQ1,EGD0000.4,0.4,WRITE,BOOL,L31,Disabled,NaN,NaN,192.168.101.110,50,FDBKS,68,1,10.0,Broadcast to 192.168.101.255,EGD TORQ REQUEST 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,G31\SS_REF_OUT,EGD0708.0,708.0,READ,REAL,NaN,Disabled,PCT,XXX,192.168.101.111,480,G31:EX4,1400,4,165.0,Broadcast,Load Commutated Inv (Static Starter) Speed Ref...
57,UDH\L31\FDBKS\HEALTHY,EGD0000.1,0.1,READ,BOOL,NaN,Disabled,NaN,NaN,NaN,320,L31:LINK,0,0,1.0,NA - Link Health,NaN
58,UDH\E31\G31\HEALTHY,EGD0000.2,0.2,READ,BOOL,NaN,Disabled,NaN,NaN,NaN,320,L31:LINK,0,0,1.0,NA - Link Health,NaN
59,UDH\G31\EX2\HEALTHY,EGD0000.3,0.3,READ,BOOL,NaN,Disabled,NaN,NaN,NaN,320,L31:LINK,0,0,1.0,NA - Link Health,NaN
